# Train set 

In [1]:
import scipy.io
import os
import numpy as np

# .mat 파일들이 저장된 디렉토리 경로
mat_files_dir = "/media/hmi/NISP_HDD_2/ECG_Dataset/Physionet_CinC_2017/Developed_Dataset/9s"
output_file = "./data/train_9s.npy"

# 저장할 데이터 리스트
all_data = []

# 자연 정렬을 위한 정렬 함수
def natural_sort_key(s):
    import re
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

# 디렉토리 내의 모든 .mat 파일을 불러와서 데이터 추출
for filename in sorted(os.listdir(mat_files_dir), key=natural_sort_key):
    if filename.endswith(".mat"):
        # MATLAB 파일 로드
        print('filename: ', filename)
        mat_data = scipy.io.loadmat(os.path.join(mat_files_dir, filename))

        # 특정 변수명을 선택해서 불러오기 (예시: 'variable_name'이 변수 이름일 경우)
        # 변수명은 저장된 .mat 파일의 실제 변수명을 사용
        variable_name = list(mat_data.keys())[-1]  # 예시로 마지막 변수를 선택
        data = mat_data[variable_name]

        # 데이터를 리스트에 추가
        all_data.append(data)

# 모든 데이터를 합쳐 하나의 배열로 변환
combined_data = np.concatenate(all_data, axis=0)

# combined_data를 .npy 파일로 저장
np.save(output_file, combined_data)

print(f"데이터가 {output_file}로 저장되었습니다.")

filename:  A00001_S1.mat
filename:  A00001_S2.mat
filename:  A00001_S3.mat
filename:  A00002_S1.mat
filename:  A00002_S2.mat
filename:  A00002_S3.mat
filename:  A00003_S1.mat
filename:  A00003_S2.mat
filename:  A00003_S3.mat
filename:  A00003_S4.mat
filename:  A00003_S5.mat
filename:  A00003_S6.mat
filename:  A00004_S1.mat
filename:  A00004_S2.mat
filename:  A00004_S3.mat
filename:  A00005_S1.mat
filename:  A00005_S2.mat
filename:  A00005_S3.mat
filename:  A00005_S4.mat
filename:  A00005_S5.mat
filename:  A00005_S6.mat
filename:  A00006_S1.mat
filename:  A00006_S2.mat
filename:  A00006_S3.mat
filename:  A00007_S1.mat
filename:  A00007_S2.mat
filename:  A00007_S3.mat
filename:  A00008_S1.mat
filename:  A00008_S2.mat
filename:  A00008_S3.mat
filename:  A00008_S4.mat
filename:  A00008_S5.mat
filename:  A00008_S6.mat
filename:  A00009_S1.mat
filename:  A00009_S2.mat
filename:  A00009_S3.mat
filename:  A00010_S1.mat
filename:  A00010_S2.mat
filename:  A00010_S3.mat
filename:  A00010_S4.mat


# Test set

In [2]:
import numpy as np
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# CSV 파일 경로
csv_file = "/media/hmi/NISP_HDD_2/ECG_Dataset/Physionet_CinC_2017/Developed_Dataset/segments_ann.csv"
output_file = "./data/labels_9s.npy"

# CSV 파일 읽기 (여기서 'label'이 레이블 열 이름이라고 가정)
df = pd.read_csv(csv_file)

class_mapping = {'N': 0, 'A': 1, 'O': 2}

# 레이블 데이터만 추출 (열 이름이 'label'인 경우)
df['Encoded Labels'] = df['Class'].map(class_mapping)

# 0, 1, 2로만 이루어진 인코딩된 레이블 추출
encoded_labels = df['Encoded Labels'].values

# numpy 파일로 저장
np.save(output_file, encoded_labels)

# 저장된 numpy 파일 확인
print(f"Encoded labels saved to {output_file}")
print(encoded_labels)

Encoded labels saved to ./data/labels_9s.npy
[0 0 0 ... 0 0 0]


# Train & Test Split

In [4]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split

# 기존 데이터 로드
np_data = np.load('./data/train_9s.npy').astype(np.float32)
label = np.load('./data/labels_9s.npy').astype(np.int32)

# Tensor 변환 및 train-test split
data = torch.tensor(np_data.reshape(np_data.shape[0], 1, np_data.shape[1]), dtype=torch.float32)
labels = torch.tensor(label, dtype=torch.long)
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.1)

# 데이터 저장
torch.save(train_data, './data/train_data_MuDANet.pt')
torch.save(val_data, './data/val_data_MuDANet.pt')
torch.save(train_labels, './data/train_labels_MuDANet.pt')
torch.save(val_labels, './data/val_labels_MuDANet.pt')

# Randomly remove 5000 Normal Samples

In [8]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split

# 랜덤 시드 고정
np.random.seed(42)

# 기존 데이터 로드
np_data = np.load('./data/train_9s.npy').astype(np.float32)
label = np.load('./data/labels_9s.npy').astype(np.int32)

# 라벨 0에 해당하는 인덱스 찾기
label_0_indices = np.where(label == 0)[0]

# 5,000개의 라벨 0 샘플을 랜덤하게 선택
remove_indices = np.random.choice(label_0_indices, 5000, replace=False)

# 선택된 샘플을 제거한 새로운 데이터와 라벨 생성
np_data_filtered = np.delete(np_data, remove_indices, axis=0)
label_filtered = np.delete(label, remove_indices, axis=0)

# Tensor 변환 및 train-test split
data = torch.tensor(np_data_filtered.reshape(np_data_filtered.shape[0], 1, np_data_filtered.shape[1]), dtype=torch.float32)
labels = torch.tensor(label_filtered, dtype=torch.long)
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.1)

# 데이터 저장
torch.save(train_data, './data/train_data_MuDANet_rem5000.pt')
torch.save(val_data, './data/val_data_MuDANet_rem5000.pt')
torch.save(train_labels, './data/train_labels_MuDANet_rem5000.pt')
torch.save(val_labels, './data/val_labels_MuDANet_rem5000.pt')


# Train data Augmentation

In [5]:
import torch

# 데이터 및 라벨 로드
train_data = torch.load('./data/train_data_MuDANet.pt')
train_labels = torch.load('./data/train_labels_MuDANet.pt')

# 클래스 1에 해당하는 샘플들의 인덱스 가져오기
class_1_indices = torch.where(train_labels == 1)[0]

# 클래스 1에 해당하는 데이터 샘플 가져오기
class_1_data = train_data[class_1_indices]
class_1_labels = train_labels[class_1_indices]

# 클래스 1의 데이터를 3배로 증강 (반복)
augmented_class_1_data = class_1_data.repeat(2, 1, 1)
augmented_class_1_labels = class_1_labels.repeat(2)

# 기존 데이터에 증강된 데이터 추가
train_data_augmented = torch.cat((train_data, augmented_class_1_data), dim=0)
train_labels_augmented = torch.cat((train_labels, augmented_class_1_labels), dim=0)

# 데이터 및 라벨을 다시 저장
torch.save(train_data_augmented, './data/train_data_MuDANet_rep.pt')
torch.save(train_labels_augmented, './data/train_labels_MuDANet_rep.pt')

print(f"Updated train_data shape: {train_data_augmented.shape}")
print(f"Updated train_labels shape: {train_labels_augmented.shape}")

Updated train_data shape: torch.Size([27876, 1, 2700])
Updated train_labels shape: torch.Size([27876])


In [6]:
train_labels = torch.load('./data/train_labels.pt')

# 클래스 1에 해당하는 샘플들의 인덱스 가져오기
class_0_indices = torch.where(train_labels == 0)[0]
class_1_indices = torch.where(train_labels == 1)[0]
class_2_indices = torch.where(train_labels == 2)[0]

print('Normal label :', len(class_0_indices))
print('AF label :', len(class_1_indices))
print('Other label :', len(class_2_indices))

Normal label : 14572
AF label : 6510
Other label : 7808


In [12]:
val_labels = torch.load('./data/val_labels.pt')

# 클래스 1에 해당하는 샘플들의 인덱스 가져오기
class_0_indices = torch.where(val_labels == 0)[0]
class_1_indices = torch.where(val_labels == 1)[0]
class_2_indices = torch.where(val_labels == 2)[0]

print('Normal label :', len(class_0_indices))
print('AF label :', len(class_1_indices))
print('Other label :', len(class_2_indices))

Normal label : 1649
AF label : 225
Other label : 854


# Remove Other Labels

In [22]:
import torch

# 데이터 및 라벨 로드
train_data = torch.load('./data/train_data.pt')
train_labels = torch.load('./data/train_labels.pt')
val_data = torch.load('./data/val_data.pt')
val_labels = torch.load('./data/val_labels.pt')

# 클래스 1에 해당하는 샘플들의 인덱스 가져오기
class_1_indices = torch.where(train_labels == 1)[0]

# 클래스 1에 해당하는 데이터 샘플 가져오기
class_1_data = train_data[class_1_indices]
class_1_labels = train_labels[class_1_indices]

# 클래스 1의 데이터를 3배로 증강 (반복)
augmented_class_1_data = class_1_data.repeat(2, 1, 1)
augmented_class_1_labels = class_1_labels.repeat(2)

# 기존 데이터에서 클래스 2에 해당하는 샘플 제거
class_2_indices = torch.where(train_labels == 2)[0]  # 클래스 2의 인덱스 가져오기
train_data = torch.index_select(train_data, 0, torch.where(train_labels != 2)[0])  # 클래스 2 제거
train_labels = train_labels[train_labels != 2]  # 클래스 2에 해당하는 라벨 제거

# val_data 및 val_labels에서 클래스 2 제거
val_data = torch.index_select(val_data, 0, torch.where(val_labels != 2)[0])
val_labels = val_labels[val_labels != 2]

# 기존 데이터에 증강된 데이터 추가
train_data_augmented = torch.cat((train_data, augmented_class_1_data), dim=0)
train_labels_augmented = torch.cat((train_labels, augmented_class_1_labels), dim=0)

# 데이터 및 라벨을 다시 저장
torch.save(train_data_augmented, './data/train_data_wo_other.pt')
torch.save(train_labels_augmented, './data/train_labels_wo_other.pt')
torch.save(val_data, './data/val_data_wo_other.pt')
torch.save(val_labels, './data/val_labels_wo_other.pt')

print(f"Updated train_data shape: {train_data_augmented.shape}")
print(f"Updated train_labels shape: {train_labels_augmented.shape}")
print(f"Updated val_data shape: {val_data.shape}")
print(f"Updated val_labels shape: {val_labels.shape}")

Updated train_data shape: torch.Size([34102, 1, 1080])
Updated train_labels shape: torch.Size([34102])
Updated val_data shape: torch.Size([1897, 1, 1080])
Updated val_labels shape: torch.Size([1897])
